In [ ]:
import os
import re
import json
import zipfile
import io
from datetime import datetime
import requests

def parse_filename(filename):
    # Example filename format: user2_disgusted_2024-06-25-10-42-40-2_pTYnOdGQ7XhP7ToFAAAF_recording_0000.png
    try:
        parts = filename.split("_")
        username = parts[0] #not needed
        emotion = parts[1]
        timestamp_str = parts[2]
        timestamp_parts = parts[2].rsplit("-", 1)  # Split at the last dash
        timestamp_str = timestamp_parts[0]
        timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d-%H-%M-%S")
        user_id = parts[3]
        return username, emotion, timestamp, user_id
    except Exception as e:
        print(f"Error parsing filename '{filename}': {e}")
        return None, None, None, None

# Main script
directory = r"C:\CHIMP-data\calibration_FA_2024-06-25_face"  # Set this to the root folder to scan
username_use = "FA"
timestamp_str_use = "2024-06-25"

url = "http://localhost:5253/managed_datasets"  # Replace with the actual URL

labels = []
metadata = []
zip_buffer = io.BytesIO()

with zipfile.ZipFile(zip_buffer, "w", zipfile.ZIP_DEFLATED) as zip_file:
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith((".png", "jpg")): 
                file_path = os.path.join(root, file)
                username, emotion, timestamp, user_id = parse_filename(file)
                
                if username and emotion and timestamp and user_id:
                    labels.append(emotion)
                    metadata.append({
                        "exp": "emotion_recognition",
                        "user": username_use,
                        "userid": user_id,
                        "timestamp": timestamp.isoformat()
                    })
                    # Add the file to the zip
                    with open(file_path, "rb") as img_file:
                        zip_file.writestr(file, img_file.read())

# Prepare the payload
timestamp_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
path_name = re.sub(r'[<>:"/\\|?*]', '', f"fer2013_{username_use}_{timestamp_str_use}_{user_id}")

files = {
    "file": (path_name + '.zip', zip_buffer.getvalue(), 'application/zip'),
    "dataset_name": (None, path_name),
    "labels": (None, json.dumps(labels)),  # JSON encode the list
    "metadata": (None, json.dumps(metadata))  # JSON encode the list
}


Error parsing filename 'Training_10118481.jpg': list index out of range
Error parsing filename 'Training_10120469.jpg': list index out of range
Error parsing filename 'Training_10131352.jpg': list index out of range
Error parsing filename 'Training_10161559.jpg': list index out of range
Error parsing filename 'Training_1021836.jpg': list index out of range
Error parsing filename 'Training_10269675.jpg': list index out of range
Error parsing filename 'Training_10278738.jpg': list index out of range
Error parsing filename 'Training_10290703.jpg': list index out of range
Error parsing filename 'Training_10295477.jpg': list index out of range
Error parsing filename 'Training_10315441.jpg': list index out of range
Error parsing filename 'Training_10316849.jpg': list index out of range
Error parsing filename 'Training_10333072.jpg': list index out of range
Error parsing filename 'Training_10334355.jpg': list index out of range
Error parsing filename 'Training_10345473.jpg': list index out of

In [20]:
# Send the request
print("Sending image zip to managed dataset:", path_name)
response = requests.post(url, files=files)

print("Response status:", response.status_code)
print("Response body:", response.text)


Sending image zip to managed dataset: fer2013_FER_2013-01-01_None
Response status: 200
Response body: {"status":"successfully uploaded dataset"}



In [ ]:
import io
import zipfile
from flask import Flask, request
from minio import Minio
from werkzeug.utils import secure_filename
import json
import psycopg2
from PIL import Image
import numpy as np
from io import BytesIO

DATASTORE_ACCESS_KEY="yZmhrURuUhaeVSUagMRa"
DATASTORE_SECRET_KEY="cnk0OxGuIgVx4La0prNaWUv7JpriCnxZfq2417ba"
DATASTORE_URI="localhost:9000"

# Initialize MinIO client
client = Minio(
    DATASTORE_URI,  # Replace with your MinIO endpoint
    access_key=DATASTORE_ACCESS_KEY,
    secret_key=DATASTORE_SECRET_KEY,
    secure=False
)
# global bucket_name
bucket_name = "manageddataset"

# if not client.bucket_exists(bucket_name):
#     client.make_bucket(bucket_name)

db_config = {
    "dbname": "chimp_database",
    "user": "chimp_user",
    "password": "chimp_password",
    "host": "localhost",  # Use the Docker host's IP if not running locally
    "port": "5432"
}



In [ ]:
from urllib.parse import urlparse

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Insert query
select_query = """
SELECT * FROM datapoints
WHERE metadata->>'user' = 'MV'
LIMIT 20
"""

cursor.execute(select_query)

# Fetch all rows
rows = cursor.fetchall()

alldata=[]
# Print each row
for row in rows:
    print(f"ID: {row[0]}, X: {row[1]}, Y: {row[2]}, Metadata: {row[3]}")
    opath = row[1]
    parsed_url = urlparse(opath)

    # Extract bucket name and object path
    path_parts = parsed_url.path.lstrip('/').split('/', 1)  # Remove leading slash and split into bucket and object path
    bucket_name = path_parts[0]  # First part is the bucket name
    object_path = path_parts[1]

    response = client.get_object(bucket_name,object_path)
    alldata.append(BytesIO(response.read()))
    response.close()
    response.release_conn()

cursor.close()




ID: 3816, X: https://localhost:9000/manageddataset/calibration_MV_2024-06-24_D8JX2GEq/user2_sad_2024-06-24-10-31-19-5_D8JX2GEq_xEzuZ0EAAAf_recording_0033.png, Y: sad, Metadata: {'exp': 'emotion_recognition', 'user': 'MV', 'userid': 'D8JX2GEq', 'timestamp': '2024-06-24T10:31:19'}
ID: 3488, X: https://localhost:9000/manageddataset/calibration_MV_2024-06-24_D8JX2GEq/user2_angry_2024-06-24-10-31-19-0_D8JX2GEq_xEzuZ0EAAAf_recording_0000.png, Y: angry, Metadata: {'exp': 'emotion_recognition', 'user': 'MV', 'userid': 'D8JX2GEq', 'timestamp': '2024-06-24T10:31:19'}
ID: 3489, X: https://localhost:9000/manageddataset/calibration_MV_2024-06-24_D8JX2GEq/user2_angry_2024-06-24-10-31-19-0_D8JX2GEq_xEzuZ0EAAAf_recording_0001.png, Y: angry, Metadata: {'exp': 'emotion_recognition', 'user': 'MV', 'userid': 'D8JX2GEq', 'timestamp': '2024-06-24T10:31:19'}
ID: 3490, X: https://localhost:9000/manageddataset/calibration_MV_2024-06-24_D8JX2GEq/user2_angry_2024-06-24-10-31-19-0_D8JX2GEq_xEzuZ0EAAAf_recording_0

In [26]:
conn.close()

In [27]:
from PIL import Image

images = []
for image_data in alldata:
    image = Image.open(image_data)  # Convert BytesIO to PIL Image
    images.append(image)

In [29]:
images[10].show()